 # Problem
We will have multiple tables in the database, and sometimes there won’t be a common “ID” or “KEY” to join them – scenarios like the following:
    • Customer information scattered across multiple tables and systems.
    • No global key to link them all together.
    • A lot of variations in names and addresses.

# Solution
This can be solved by applying text similarity functions on the demographic’s columns like the first name, last name, address, etc. And based on the similarity score on a few common columns, we can decide either the record pair is a match or not a match

# Deduplication in the same table

  Created wheel for jellyfish: filename=jellyfish-0.8.2-py3-none-any.whl size=10424 sha256=ad7eb6a3a68597b1a719f6386cb4fb9ac507ef85828079d6f030b63ed21b7ec4
  Stored in directory: c:\users\baban\appdata\local\pip\cache\wheels\12\9e\c1\46af1db49a60a4adf555196c0ead132e4a36a2bd69b078e740
Successfully built jellyfish


In [2]:
# Import package
import recordlinkage
#For this demo let us use the inbuilt dataset from recordlinkage library

In [3]:
#import data set
from recordlinkage.datasets import load_febrl1
#create a dataframe - dfa
dfA = load_febrl1()
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


In [4]:
#  let’s build the pairs using the first name as the blocking index.
indexer = recordlinkage.BlockIndex(on='given_name')
pairs = indexer.index(dfA)
print (len(pairs))

2082


In [ ]:
# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()
compare_cl.string('given_name', 'given_name',method='jarowinkler', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1',method='jarowinkler',label='address_1')
features = compare_cl.compute(pairs, dfA)
features.sample(5)

In [ ]:
# select all the features except for given_name since its our blocking key
features1 = features[['suburb','state','surname','date_of_birth','address_1']]
# Unsupervised learning – probabilistic
ecm = recordlinkage.ECMClassifier()
result_ecm = ecm.learn((features1).astype(int),return_type ='series')
result_ecm

# deduplication in multiple tables

In [6]:
from recordlinkage.datasets import load_febrl4
dfA, dfB = load_febrl4()
dfB.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


In [ ]:
indexer = recordlinkage.BlockIndex(on='given_name')
pairs = indexer.index(dfA, dfB)

In [ ]:
compare_cl = recordlinkage.Compare()
compare_cl.string('given_name', 'given_name',method='jarowinkler',label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler',label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1',method='jarowinkler',label='address_1')
features = compare_cl.compute(pairs, dfA, dfB)
features.head(10)

"""
ECM – classifier : is an unsupervised learning method to calculate the probability that the record is a match.
""" 

In [ ]:
# select all the features except for given_name since its our blocking key
features1 = features[['suburb','state','surname','date_of_birth','address_1']]
# unsupervised learning - probablistic
ecm = recordlinkage.ECMClassifier()
result_ecm = ecm.learn((features1).astype(int),return_type ='series')
result_ecm